In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.5.2-bin-hadoop3'

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from datetime import datetime

Task 1

In [3]:
appName = "Big Data Analytics"
master = "local"

conf = pyspark.SparkConf()\
.set('spark.driver.host','127.0.0.1')\
.setAppName(appName)\
.setMaster(master)

sc = SparkContext.getOrCreate(conf=conf)

sqlContext = SQLContext(sc)

spark = sqlContext.sparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

C:\spark\spark-3.5.2-bin-hadoop3\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
from pyspark.sql.functions import lit, monotonically_increasing_id
df_list = list()

## read in all data for male players
for i in range(15, 23):
    file_name = "data\\players_" + str(i) + ".csv"
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    year = 2000 + i
    gender = "Male"
    df = df.withColumn("year", lit(year))
    df = df.withColumn("gender", lit(gender))
    df_list.append(df)
    
## read in all data for female players
for i in range(16, 23):
    file_name = "data\\female_players_" + str(i) + ".csv"
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    year = 2000 + i
    gender = "Female"
    df = df.withColumn("year", lit(year))
    df = df.withColumn("gender", lit(gender))
    df_list.append(df)
    
## merge all data into one dataframe
df_merged = df_list[0]
for df in df_list[1:]:
    df_merged = df_merged.union(df)
    
## create new column to storage unique id for each piece of data 
df_merged = df_merged.withColumn("record_id", monotonically_increasing_id())


In [5]:
## write data into PostgreDB
properties = {
    "user": "postgres",
    "password": "QQwa43420024420-",
    "driver": "org.postgresql.Driver"
}
url = "jdbc:postgresql://localhost:5432/postgres"
table = "fifa.players"
df_merged.write \
    .jdbc(url=url, table=table, mode="overwrite", properties=properties)

code to create schema fifa and table fifa.players refer to task1.sql

In [6]:
## read data from PostgreDB
db_properties={}
db_properties['username'] = "postgres"
db_properties['password'] = "QQwa43420024420-"
db_properties['url'] = "jdbc:postgresql://localhost:5432/postgres"
db_properties['driver'] = "org.postgresql.Driver"
db_properties['table'] = "fifa.players"

df_read = sqlContext.read.format("jdbc")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.load()

## Only analyzing data for male players
df_read = df_read.filter(df_read.gender == "Male")


Task 2

In [7]:
from pyspark.sql.functions import col, desc, asc

## Task 2.1
def get_clubs_with_most_players(year, n_club, ending_year):
    df_casted = df_read.withColumn("club_contract_valid_until_int", col("club_contract_valid_until").cast("integer"))
    df_filtered = df_casted.filter((df_casted.year == year) & (df_casted.club_contract_valid_until_int >= ending_year))
    df_grouped = df_filtered.groupBy("club_name").count()
    df_ordered = df_grouped.orderBy(desc("count"))
    df_limited = df_ordered.limit(n_club)
    print(f"the {n_club} clubs with most players in year {year} whose contract ending in or after year {ending_year}")
    df_limited.show()

## Task 2.2 parameter order is in ["highest", "lowest"]
def get_clubs_with_highest_or_lowest_average_age(year, n_club, order):
    original_n = n_club
    df_filtered = df_read.filter(df_read.year == year)
    df_grouped = df_filtered.groupBy("club_name").avg("age")
    if order == "lowest":
        df_ordered = df_grouped.orderBy(asc("avg(age)"))
    elif order == "highest":
        df_ordered = df_grouped.orderBy(desc("avg(age)"))
    if n_club < df_ordered.count():
        while 1:
            if df_ordered.limit(n_club).collect()[-1]["avg(age)"] == df_ordered.limit(n_club+1).collect()[-1]["avg(age)"]:
                n_club += 1
            else:
                break
    df_limited = df_ordered.limit(n_club)
    print(f"the {original_n} clubs with {order} average ages for players in year {year}")
    df_limited.show()

## Task 2.3
def get_most_popular_nationality(year):
    df_filtered = df_read.filter(df_read.year == year)
    df_grouped = df_filtered.groupBy("nationality_name").count()
    df_ordered = df_grouped.orderBy(desc("count"))
    df_limited = df_ordered.limit(1)
    print(f"Most Popular Nationality in Year {year}")
    df_limited.show()

In [8]:
## Test task 2.1 get_clubs_with_most_players(year, n_club, ending_year)
get_clubs_with_most_players(2020, 5, 2024)

the 5 clubs with most players in year 2020 whose contract ending in or after year 2024
+-------------------+-----+
|          club_name|count|
+-------------------+-----+
|   Deportes Iquique|   12|
|Patriotas Boyacá FC|   12|
|          Al Ain FC|   11|
|  Alianza Petrolera|   11|
|     Atlético Huila|   11|
+-------------------+-----+



In [9]:
## Test task 2.2 get_clubs_with_highest_or_lowest_average_age(year, n_club, order)
get_clubs_with_highest_or_lowest_average_age(2020, 3, "highest")

the 3 clubs with highest average ages for players in year 2020
+--------------------+--------+
|           club_name|avg(age)|
+--------------------+--------+
|           Fortaleza|    32.6|
|            Cruzeiro|    31.6|
|Club Athletico Pa...|    31.4|
|            Botafogo|    31.4|
|Associação Chapec...|    31.4|
+--------------------+--------+



In [10]:
## Test task 2.3 get_most_popular_nationality(year)
for year in range(2015, 2023):
    get_most_popular_nationality(year)

Most Popular Nationality in Year 2015
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1627|
+----------------+-----+

Most Popular Nationality in Year 2016
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1519|
+----------------+-----+

Most Popular Nationality in Year 2017
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1627|
+----------------+-----+

Most Popular Nationality in Year 2018
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1633|
+----------------+-----+

Most Popular Nationality in Year 2019
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1625|
+----------------+-----+

Most Popular Nationality in Year 2020
+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1670|
+----------------+-----+

Most Popular Nat

Task 3

In [11]:
# data preprocession
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# define all useful features
feature_cols_int = ["weak_foot", "skill_moves", "international_reputation", "pace", "shooting", "passing", "dribbling", "defending", "physic", "attacking_crossing", "attacking_finishing", "attacking_heading_accuracy", "attacking_short_passing", "attacking_volleys", "skill_dribbling", "skill_curve", "skill_fk_accuracy", "skill_long_passing", "skill_ball_control", "movement_acceleration", "movement_sprint_speed", "movement_agility", "movement_reactions", "movement_balance", "power_shot_power", "power_jumping", "power_stamina", "power_strength", "power_long_shots", "mentality_aggression", "mentality_interceptions", "mentality_positioning", "mentality_vision", "mentality_penalties", "defending_marking_awareness", "defending_standing_tackle", "defending_sliding_tackle", "goalkeeping_diving", "goalkeeping_handling", "goalkeeping_kicking", "goalkeeping_positioning", "goalkeeping_reflexes", "goalkeeping_speed"]
feature_cols_to_onehot = ["work_rate", "mentality_composure", "ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
outcome_col = ["overall"]
df = df_read.select(feature_cols_int + feature_cols_to_onehot + outcome_col)

In [12]:
# calculate the ratio of null value for each feature
df_count = df.count()
null_counts_df = df.select([(count(when(isnan(col(c)) | col(c).isNull(), c))/df_count*100).alias(c) \
                        for c in df.columns])
null_counts_df.show(vertical=True)

-RECORD 0-----------------------------------------
 weak_foot                   | 0.0                
 skill_moves                 | 0.0                
 international_reputation    | 0.0                
 pace                        | 11.114239261256062 
 shooting                    | 11.114239261256062 
 passing                     | 11.114239261256062 
 dribbling                   | 11.114239261256062 
 defending                   | 11.114239261256062 
 physic                      | 11.114239261256062 
 attacking_crossing          | 0.0                
 attacking_finishing         | 0.0                
 attacking_heading_accuracy  | 0.0                
 attacking_short_passing     | 0.0                
 attacking_volleys           | 0.0                
 skill_dribbling             | 0.0                
 skill_curve                 | 0.0                
 skill_fk_accuracy           | 0.0                
 skill_long_passing          | 0.0                
 skill_ball_control          | 

In [13]:
# drop "goalkeeping_speed" because tremendous missing values, then drop the rest data if there is missing value in each row
df = df.drop("goalkeeping_speed")
feature_cols_int.remove("goalkeeping_speed")
df = df.dropna()

In [14]:
# encode all features and generate final df
for c in (feature_cols_int + outcome_col):
    df = df.withColumn(c+"_encoded", col(c).cast(DoubleType()))
feature_cols_int_to_double = [x+"_encoded" for x in (feature_cols_int + outcome_col)]
feature_cols_index = [x+"_index" for x in feature_cols_to_onehot]
feature_cols_onehot = [x+"_encoded" for x in feature_cols_to_onehot]
indexer = StringIndexer(inputCols=feature_cols_to_onehot, outputCols=feature_cols_index, handleInvalid="keep")
df_index_encoded = indexer.fit(df).transform(df)
encoder = OneHotEncoder(inputCols=feature_cols_index, outputCols=feature_cols_onehot, handleInvalid="keep")
df_onehot_encoded = encoder.fit(df_index_encoded).transform(df_index_encoded)
assembler = VectorAssembler(inputCols=feature_cols_int_to_double + feature_cols_onehot, outputCol="features", handleInvalid="keep")
df_assembled = assembler.transform(df_onehot_encoded)
df_final = df_assembled.select(["features", "overall_encoded"])

In [15]:
# example cases of df_final
df_final.show(5, vertical=True)

-RECORD 0-------------------------------
 features        | (7993,[0,1,2,3,4,... 
 overall_encoded | 66.0                 
-RECORD 1-------------------------------
 features        | (7993,[0,1,2,3,4,... 
 overall_encoded | 94.0                 
-RECORD 2-------------------------------
 features        | (7993,[0,1,2,3,4,... 
 overall_encoded | 93.0                 
-RECORD 3-------------------------------
 features        | (7993,[0,1,2,3,4,... 
 overall_encoded | 92.0                 
-RECORD 4-------------------------------
 features        | (7993,[0,1,2,3,4,... 
 overall_encoded | 92.0                 
only showing top 5 rows



In [16]:
# there are only 50 different labels, so that the problem can be regarded as classification problem
N_labels = df_final.select("overall_encoded").distinct().count()
print(f"there are {N_labels} distinct labels")

there are 50 distinct labels


In [17]:
# Only randomly choose 10000 rows as the dataset for further training and testing to save time
df_omit = df_final.sample(withReplacement=False, fraction=1.0, seed=2024).limit(10000)
train_data, test_data = df_omit.randomSplit([0.8, 0.2])

In [18]:
# Logistic Regression on spark
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
regParam_range = [0.01, 0.1]
maxIter_range = [10, 30, 50]
tuning_list = []
for regParam in regParam_range:
    for maxIter in maxIter_range:
        lr = LogisticRegression(featuresCol="features", labelCol="overall_encoded", regParam=regParam, maxIter=maxIter)
        lr_model = lr.fit(train_data)
        predictions = lr_model.transform(test_data)
        evaluator = MulticlassClassificationEvaluator(labelCol="overall_encoded", predictionCol="prediction", metricName="accuracy")
        accuracy = evaluator.evaluate(predictions)
        tuning_list.append((regParam, maxIter, accuracy))
tuning_list = sorted(tuning_list, key=lambda x:x[-1])
best_tuple = tuning_list[-1]
best_regParam = best_tuple[0]
best_maxIter = best_tuple[1]
best_accuracy = best_tuple[2]
print(f"when regParam={best_regParam}, maxIter={best_maxIter}, there is highest accuracy for Logistic Regression: {100*best_accuracy}%")

when regParam=0.01, maxIter=50, there is highest accuracy for Logistic Regression: 79.62037962037962%


In [19]:
# Decision Classifier on Spark
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
maxDepth_range = [10, 30]
impurity_range = ["gini", "entropy"]

tuning_list = []
for impurity in impurity_range:
    for maxDepth in maxDepth_range:
        dt = DecisionTreeClassifier(featuresCol="features", labelCol="overall_encoded", maxDepth=maxDepth, impurity=impurity)
        dt_model = dt.fit(train_data)
        predictions = dt_model.transform(test_data)
        evaluator = MulticlassClassificationEvaluator(labelCol="overall_encoded", predictionCol="prediction", metricName="accuracy")
        accuracy = evaluator.evaluate(predictions)
        tuning_list.append((impurity, maxDepth, accuracy))
tuning_list = sorted(tuning_list, key=lambda x:x[-1])
best_tuple = tuning_list[-1]
best_impurity = best_tuple[0]
best_maxDepth = best_tuple[1]
best_accuracy = best_tuple[2]
print(f"when impurity={best_impurity}, maxDepth={best_maxDepth}, there is highest accuracy for Decision Tree: {100*best_accuracy}%")
        
        

when impurity=entropy, maxDepth=10, there is highest accuracy for Decision Tree: 100.0%


In [20]:
# transform data from spark to tensor
import torch 
from torch import nn
from torch.utils.data import Dataset, DataLoader
train_data_pd = train_data.toPandas()
test_data_pd = test_data.toPandas()
X_train = torch.Tensor(train_data_pd["features"])
y_train = torch.Tensor(train_data_pd["overall_encoded"])
X_test = torch.Tensor(test_data_pd["features"])
y_test = torch.Tensor(test_data_pd["overall_encoded"])

In [21]:
# pytorch neural network 1 without hidden layer
import torch 
from torch import nn
from torch.utils.data import Dataset, DataLoader
N_labels = df_final.select("overall_encoded").distinct().count()
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, idx):
        return (self.x[idx],self.y[idx])

train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

class myModel1(nn.Module):
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(input_dims, output_dims),
        )
    def forward(self, X):
            return self.seq(X)

In [22]:
# model1 structure
model = myModel1(train_dataset.x.shape[1], 1)
print(model)

myModel1(
  (seq): Sequential(
    (0): Linear(in_features=7993, out_features=1, bias=True)
  )
)


In [23]:
lr_range = [0.005, 0.05, 0.1]
batch_size_range = [500, 2000]
N_epochs = 50
tuning_list = []
for lr in lr_range:
    for batch_size in batch_size_range:
        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle =True)
        model = myModel1(train_dataset.x.shape[1], 1)
        optimizer = torch.optim.Adam(model.parameters(), lr = lr)
        loss_fn = nn.MSELoss()
        for epoch in range(N_epochs):
            for batch_id, (x_batch, y_batch) in enumerate(train_dataloader):
                predictions = model(x_batch)
                loss = loss_fn(predictions, y_batch.reshape(-1, 1))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        pred = model(X_test)
        accuracy = 100 * torch.sum(torch.round(pred) == y_test.reshape(-1, 1)) / len(y_test)
        tuning_list.append((model, lr, batch_size, accuracy))

tuning_list = sorted(tuning_list, key=lambda x:x[-1])
best_tuple = tuning_list[-1]
best_lr = best_tuple[1]
best_batch_size = best_tuple[2]
best_accuracy = best_tuple[3]
print(f"when lr={best_lr}, batch_size={best_batch_size}, there is highest accuracy for Model1: {best_accuracy}%")

when lr=0.1, batch_size=500, there is highest accuracy for Model1: 41.60839080810547%


In [24]:
# pytorch neural network 2 with 3 hidden layers, each layer with 128 neurons
class myModel2(nn.Module):
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(input_dims, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dims)
            
        )
    def forward(self, X):
            return self.seq(X)
        

In [25]:
# model2 structure
model = myModel2(train_dataset.x.shape[1], 1)
print(model)

myModel2(
  (seq): Sequential(
    (0): Linear(in_features=7993, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [26]:
lr_range = [0.005, 0.05, 0.1]
batch_size_range = [500, 2000]
N_epochs = 50
tuning_list = []
for lr in lr_range:
    for batch_size in batch_size_range:
        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle =True)
        model = myModel2(train_dataset.x.shape[1], 1)
        optimizer = torch.optim.Adam(model.parameters(), lr = lr)
        loss_fn = nn.MSELoss()
        for epoch in range(N_epochs):
            for batch_id, (x_batch, y_batch) in enumerate(train_dataloader):
                predictions = model(x_batch)
                loss = loss_fn(predictions, y_batch.reshape(-1, 1))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        pred = model(X_test)
        accuracy = 100 * torch.sum(pred.int() == y_test.reshape(-1, 1).int()) / len(y_test)
        tuning_list.append((model, lr, batch_size, accuracy))

tuning_list = sorted(tuning_list, key=lambda x:x[-1])
best_tuple = tuning_list[-1]
best_lr = best_tuple[1]
best_batch_size = best_tuple[2]
best_accuracy = best_tuple[3]
print(f"when lr={best_lr}, batch_size={best_batch_size}, there is highest accuracy: {best_accuracy}%")

when lr=0.005, batch_size=500, there is highest accuracy: 33.61638259887695%


Task 4

refer to Task4.ipynb